In [1]:
with open('/content/the-verdict.txt','r',encoding='utf-8') as f:
  raw_data = f.read()
  print("total number of characters",len(raw_data))
  print(raw_data[:100])

total number of characters 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [2]:
import re
text = 'Hello, world. This, is a test. ? ?'
result = re.split(r'([,.]|\s)',text)
print(result)


['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '', ' ', '?', ' ', '?']


In [3]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.', '?', '?']


In [4]:
if " ".strip():
  print("hi")

In [5]:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)',raw_data)
preprocessed = [item for item in preprocessed if item.strip()]

In [22]:
preprocessed[:30]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was',
 'no',
 'great',
 'surprise',
 'to',
 'me',
 'to',
 'hear',
 'that',
 ',',
 'in']

Now we will firstly remove duplicates by set DataStructure then Convert it to list because set return Dictinory next sort them with sort function

In [6]:
#remove Duplicates
non_duplicates_words = set(preprocessed)
#convert to list
word_list = list(non_duplicates_words)
#sort values
all_words = sorted(word_list)

In [8]:
#adding special tokens
all_words.extend(['<|unk|>','<|endoftext|>'])

In [9]:
print(len(raw_data), len(preprocessed), len(all_words))

20479 4649 1161


next doing vocabulary like Word:ID dict



In [10]:
vocab = {word:idx for idx,word in enumerate(all_words)}

In [11]:
for idx,word in enumerate(vocab.items()):
  if idx > 10:
    break
  print(word)

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)


Now we will build a simple tokenizer class with  2 features:
1. from Words to IDs (Encoding)
2. from IDs to Words (Decoding)   




In [16]:
class SimpleTokenizer:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_Str = {idx:words for words,idx in vocab.items()}
    def encode(self,text):
        preprocessed = re.split('([,.?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else '<|unk|>'
            for item in preprocessed
        ]
        ids  = [self.str_to_int[token] for token in preprocessed]
        return ids
    def decode(self,ids):
        text = "".join([self.int_to_Str[idx] for idx in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [17]:
tokenizer = SimpleTokenizer(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [18]:
tokenizer.encode(text)

[1159,
 5,
 362,
 1155,
 642,
 1000,
 10,
 1160,
 57,
 1013,
 981,
 1009,
 738,
 1013,
 1159,
 7]

In [19]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>,doyouliketea?<|endoftext|>Inthesunlitterracesofthe<|unk|>.'

Now we will use gpt2 tokenizer

In [23]:
%pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00


In [24]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [25]:
tokenizer = tiktoken.get_encoding("gpt2")

In [26]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [27]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.
